# Assignment 1

Task 1

In [ ]:
# Configs
from rdflib import Graph, Namespace, RDF, RDFS, OWL, Literal, XSD
from rdflib.namespace import FOAF
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os

OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
DATA_DIR = "data"
SCHEMA_FILE = os.path.join(OUTPUT_DIR, "schema.ttl")
BASE_URI = "http://kg-course.io/food-nutrition/"

RECIPES_CSV = os.path.join(DATA_DIR, "Recipes.csv")
NUTRITION_CSV = os.path.join(DATA_DIR, "Nutrition.csv")
RESTAURANTS_CSV = os.path.join(DATA_DIR, "Restaurants.csv")
REVIEWS_TXT = os.path.join(DATA_DIR, "Reviews.txt")

In [ ]:
def create_schema():
    g = Graph()
    
    BASE = Namespace(BASE_URI)
    SCHEMA = Namespace("http://schema.org/")
    
    g.bind("", BASE)
    g.bind("schema", SCHEMA)
    g.bind("rdf", RDF)
    g.bind("rdfs", RDFS)
    g.bind("owl", OWL)
    g.bind("xsd", XSD)
    g.bind("foaf", FOAF)
    
    classes = [
        (BASE.Recipe, SCHEMA.Recipe, "Recipe"),
        (BASE.Nutrition, SCHEMA.NutritionInformation, "Nutrition"),
        (BASE.Restaurant, SCHEMA.Restaurant, "Restaurant"),
        (BASE.Review, SCHEMA.Review, "Review"),
        (BASE.Ingredient, SCHEMA.Ingredient, "Ingredient"),
        (BASE.Author, SCHEMA.Person, "Author"),
        (BASE.Cuisine, None, "Cuisine")
    ]
    
    for cls, schema_cls, label in classes:
        g.add((cls, RDF.type, RDFS.Class))
        g.add((cls, RDF.type, OWL.Class))
        g.add((cls, RDFS.label, Literal(label, lang="en")))
        if schema_cls:
            g.add((cls, RDFS.subClassOf, schema_cls))
    
    # For integration and SPARQL queries (connections)
    properties = [
        # Recipe
        (BASE.hasNutrition, BASE.Recipe, BASE.Nutrition),
        (BASE.hasReview, BASE.Recipe, BASE.Review),
        (BASE.hasCuisine, BASE.Recipe, BASE.Cuisine),
        (BASE.hasIngredient, BASE.Recipe, BASE.Ingredient),
        
        # Restaurant
        (BASE.servesCuisine, BASE.Restaurant, BASE.Cuisine),
        
        # Review
        (BASE.mentionsIngredient, BASE.Review, BASE.Ingredient),
    ]
    
    for prop, domain, range_cls in properties:
        g.add((prop, RDF.type, RDF.Property))
        g.add((prop, RDF.type, OWL.ObjectProperty))
        g.add((prop, RDFS.domain, domain))
        g.add((prop, RDFS.range, range_cls))
    
    # For attributes
    datatype_props = [
        (BASE.hasAggregateSentiment, BASE.Recipe, XSD.float),
        (BASE.hasSentimentScore, BASE.Review, XSD.float),
        (BASE.hasSentimentLabel, BASE.Review, XSD.string),
        (BASE.hasOnlineDelivery, BASE.Restaurant, XSD.boolean),
        (BASE.hasTableBooking, BASE.Restaurant, XSD.boolean),
        (BASE.averageCostForTwo, BASE.Restaurant, XSD.float),
    ]
    
    for prop, domain, range_type in datatype_props:
        g.add((prop, RDF.type, RDF.Property))
        g.add((prop, RDF.type, OWL.DatatypeProperty))
        g.add((prop, RDFS.domain, domain))
        g.add((prop, RDFS.range, range_type))
    
    return g

schema_graph = create_schema()
print(f"{len(schema_graph)} triples")

In [ ]:
def create_diagram():
    G = nx.DiGraph()
    
    nodes = ['Recipe', 'Nutrition', 'Restaurant', 'Review', 'Ingredient', 'Author', 'Cuisine']
    G.add_nodes_from(nodes)
    
    edges = [ # With labels
        ('Recipe', 'Nutrition', 'hasNutrition'),
        ('Recipe', 'Review', 'hasReview'),
        ('Recipe', 'Cuisine', 'hasCuisine'),
        ('Recipe', 'Ingredient', 'hasIngredient'),
        ('Restaurant', 'Cuisine', 'servesCuisine'),
        ('Review', 'Recipe', 'itemReviewed'),
        ('Review', 'Author', 'author'),
        ('Review', 'Ingredient', 'mentionsIngredient'),
    ]
    
    for src, tgt, label in edges:
        G.add_edge(src, tgt, label=label)
    
    plt.figure(figsize=(14, 10))
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    colors = {
        'Recipe': '#3498db', 'Restaurant': '#3498db', # Core entities
        'Nutrition': '#2ecc71', # Structured data
        'Review': '#e74c3c', # Unstructured data
        'Ingredient': '#f39c12', # Extracted entities
        'Author': '#9b59b6', # Person
        'Cuisine': '#1abc9c' # Bridge entity
    }
    node_colors = [colors[n] for n in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=3000, alpha=0.9)
    nx.draw_networkx_labels(G, pos, font_size=11, font_weight='bold')
    nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True, arrowsize=20, width=2, connectionstyle='arc3,rad=0.1')
    
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=9, bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
    plt.title('KG schema\nfood & nutrition', fontsize=14, fontweight='bold', pad=20)
    
    # Legend
    legend = [
        mpatches.Patch(color='#3498db', label='Core entities'),
        mpatches.Patch(color='#2ecc71', label='Structured data'),
        mpatches.Patch(color='#e74c3c', label='Unstructured data'),
        mpatches.Patch(color='#f39c12', label='Extracted entities'),
        mpatches.Patch(color='#9b59b6', label='Person'),
        mpatches.Patch(color='#1abc9c', label='Bridge entity')
    ]
    plt.legend(handles=legend, loc='upper left', fontsize=9)
    plt.axis(False)
    plt.tight_layout()
    
    # Save
    diagram_file = os.path.join(OUTPUT_DIR, "schema_diagram.png")
    plt.savefig(diagram_file, dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    print(f"Diagram: {diagram_file}")

create_diagram()

In [ ]:
schema_graph.serialize(destination=SCHEMA_FILE, format="turtle")

print(f"Schema location: {SCHEMA_FILE}")
print(f"Triples: {len(schema_graph)}")

In [ ]:
# Display schema statistics
print("TASK 1")

classes = list(schema_graph.subjects(RDF.type, RDFS.Class))
properties = list(schema_graph.subjects(RDF.type, RDF.Property))

print(f"\nClasses: {len(classes)}")
for cls in classes:
    label = schema_graph.value(cls, RDFS.label)
    print(f"  - {label}")

print(f"\nProperties: {len(properties)}\n")

print("Recipe <-> Restaurant via Cuisine")
print("Recipe <-> Nutrition (1:1 linking)")
print("Recipe <-> Review (sentiment)")
print("Ingredient <-> Wikidata (external KG)")

Task 2

In [ ]:
import pandas as pd
import hashlib
from rdflib import Graph, Namespace, RDF, Literal, XSD
import os
LIMIT = 10000
BASE = Namespace(BASE_URI)
SCHEMA = Namespace("http://schema.org/")

In [ ]:
def clean(s):
    return "" if pd.isna(s) else str(s).strip()

def extract_cuisines(text):
    if not text:
        return []
    text = text.lower()
    cuisines = ['italian', 'chinese', 'indian', 'mexican', 'japanese', 'thai', 'french', 'greek', 'spanish', 'korean', 'vietnamese', 'american', 'mediterranean', 'asian', 'european']
    return [c for c in cuisines if c in text]

def load_csv(filepath, nrows=None):
    df = pd.read_csv(filepath, nrows=nrows, sep=';', on_bad_lines='skip') # Skip bad lines and use semicolon delimiter
    if len(df.columns) > 1:
        return df

# Load CSVs
recipes_df = load_csv(RECIPES_CSV, nrows=LIMIT)
print(f"{len(recipes_df)} recipes")
nutrition_df = load_csv(NUTRITION_CSV, nrows=LIMIT)
print(f"{len(nutrition_df)} nutrition records")
restaurants_df = load_csv(RESTAURANTS_CSV, nrows=LIMIT)
print(f"{len(restaurants_df)} restaurants")

In [ ]:
# Initialize graph with schema for converting to RDF
g = Graph()
g.parse(SCHEMA_FILE, format="turtle")

recipe_name_to_uri = {}
cuisine_uris = {}
cuisine_counter = 0
print(f"{len(g)} triples from schema")

In [ ]:
# Convert Recipes
def find_col(df, search_terms):
    
    if isinstance(search_terms, str):
        search_terms = [search_terms]
    
    for col in df.columns:
        for term in search_terms:
            if term.lower() in col.lower():
                return col
    
    # Return first column as fallback
    return df.columns[0]

cuisine_counter = 0

for idx, row in recipes_df.iterrows():
    # print(idx) # debug
    recipe_uri = BASE[f"recipe_{idx}"]
    
    g.add((recipe_uri, RDF.type, BASE.Recipe)) # Add type to created URI
    
    # Add properties
    name = clean(row.get('Name'))
    if name:
        g.add((recipe_uri, SCHEMA.name, Literal(name)))
        recipe_name_to_uri[name] = recipe_uri
    
    if pd.notna(row.get('CookTime')):
        g.add((recipe_uri, SCHEMA.cookTime, Literal(str(row['CookTime']))))
    
    if pd.notna(row.get('PrepTime')):
        g.add((recipe_uri, SCHEMA.prepTime, Literal(str(row['PrepTime']))))
    
    if pd.notna(row.get('DatePublished')):
        g.add((recipe_uri, SCHEMA.datePublished, Literal(str(row['DatePublished']))))
    
    if pd.notna(row.get('RecipeCategory')):
        g.add((recipe_uri, SCHEMA.recipeCategory, Literal(clean(row['RecipeCategory']))))
    
    if pd.notna(row.get('Keywords')):
        g.add((recipe_uri, SCHEMA.keywords, Literal(clean(row['Keywords']))))
    
    if pd.notna(row.get('RecipeIngredientParts')):
        g.add((recipe_uri, SCHEMA.recipeIngredient, Literal(clean(row['RecipeIngredientParts']))))
    
    # Extract cuisines
    category = clean(row.get('RecipeCategory', '')) + ' ' + clean(row.get('Keywords', ''))
    for cuisine in extract_cuisines(category):
        if cuisine not in cuisine_uris: # Check
            cuisine_uri = BASE[f"cuisine_{cuisine_counter}"]
            g.add((cuisine_uri, RDF.type, BASE.Cuisine))
            g.add((cuisine_uri, SCHEMA.name, Literal(cuisine)))
            cuisine_uris[cuisine] = cuisine_uri # Store
            cuisine_counter += 1
        else:
            cuisine_uri = cuisine_uris[cuisine] # Get
        g.add((recipe_uri, BASE.hasCuisine, cuisine_uri)) # Link recipe to cuisine

        
        g.add((recipe_uri, BASE.hasCuisine, cuisine_uri)) # Link recipe to cuisine

print(f"{len(recipes_df)} Recipes")
print(f"{len(g)} triples")

# print(f"Built lookup for {len(recipe_name_to_uri)} recipe names")

In [ ]:
# Convert nutrition and link to recipes

print("Nutrition cols:", nutrition_df.columns.tolist())
linked = 0
for idx, row in nutrition_df.iterrows():
    # Create URI
    name_col = [col for col in nutrition_df.columns if 'name' in col.lower()][0]
    name = clean(row[name_col])
    # nutrition_uri = BASE[f"nutrition_{nutrition_counter}"]
    nutrition_uri = BASE[f"nutrition_{idx}"]
    
    g.add((nutrition_uri, RDF.type, BASE.Nutrition))
    
    # Link to recipe
    recipe_uri = recipe_name_to_uri.get(name)
    if recipe_uri:
        g.add((recipe_uri, BASE.hasNutrition, nutrition_uri))
        linked += 1
    
    # Add nutritional values
    for field, predicate in [
        ('Calories', SCHEMA.calories),
        ('FatContent', SCHEMA.fatContent),
        ('SaturatedFatContent', SCHEMA.saturatedFatContent),
        ('CholesterolContent', SCHEMA.cholesterolContent),
        ('SodiumContent', SCHEMA.sodiumContent),
        ('CarbohydrateContent', SCHEMA.carbohydrateContent),
        ('FiberContent', SCHEMA.fiberContent),
        ('SugarContent', SCHEMA.sugarContent),
        ('ProteinContent', SCHEMA.proteinContent)
    ]:
        if pd.notna(row.get(field)):
            g.add((nutrition_uri, predicate, Literal(float(row[field]), datatype=XSD.float)))

print(f"Converted {len(nutrition_df)} nutrition records")
print(f"Linked {linked} to recipes")
print(f"Graph count: {len(g)} triples")

In [ ]:
# Convert Restaurants
print("Recipe columns:", recipes_df.columns.tolist())
print("Nutrition columns:", nutrition_df.columns.tolist())
print("Restaurant columns:", restaurants_df.columns.tolist())
def find_col(df, search_terms):
    if isinstance(search_terms, str):
        search_terms = [search_terms]
    
    for col in df.columns:
        for term in search_terms:
            if term.lower() in col.lower():
                return col
    return df.columns[0]

restaurants_df.columns = restaurants_df.columns.str.strip() # Normalize column names

rest_id_col = find_col(restaurants_df, ['Restaurant ID', 'ID', 'RestaurantID'])
name_col = find_col(restaurants_df, ['Restaurant Name', 'Name', 'RestaurantName'])
city_col = find_col(restaurants_df, ['City', 'Locality', 'Location'])
cuisines_col = find_col(restaurants_df, ['Cuisines', 'Cuisine'])
cost_col = find_col(restaurants_df, ['Average cost of two in USD', 'Average Cost for two', 'Average Cost for Two'])

# print("Names of cols:")
# print("  ID      :", rest_id_col)
# print("  Name    :", name_col)
# print("  City    :", city_col)
# print("  Cuisines:", cuisines_col)

for idx, row in restaurants_df.iterrows():
    restaurant_uri = BASE[f"restaurant_{idx}"]

    g.add((restaurant_uri, RDF.type, BASE.Restaurant))

    # Name
    if pd.notna(row.get(name_col)):
        g.add((restaurant_uri, SCHEMA.name, Literal(clean(row.get(name_col)))))

    # Address
    if pd.notna(row.get('Address')):
        g.add((restaurant_uri, SCHEMA.address, Literal(clean(row.get('Address')))))

    # City using detected city column
    if pd.notna(row.get(city_col)):
        g.add((restaurant_uri, SCHEMA.addressLocality, Literal(clean(row.get(city_col)))))

    # Country
    if pd.notna(row.get('Country')):
        g.add((restaurant_uri, SCHEMA.addressCountry, Literal(clean(row.get('Country')))))

    # Geo coordinates
    if pd.notna(row.get('Latitude')) and pd.notna(row.get('Longitude')):
        lat, lon = float(row.get('Latitude')), float(row.get('Longitude'))
        if lat != 0 and lon != 0:
            geo_uri = BASE[f"geo_{idx}"]
            g.add((geo_uri, RDF.type, SCHEMA.GeoCoordinates))
            g.add((geo_uri, SCHEMA.latitude, Literal(lat, datatype=XSD.float)))
            g.add((geo_uri, SCHEMA.longitude, Literal(lon, datatype=XSD.float)))
            g.add((restaurant_uri, SCHEMA.geo, geo_uri))

    # Aggregate rating
    if pd.notna(row.get('Aggregate rating')):
        g.add((restaurant_uri, SCHEMA.aggregateRating,
                   Literal(float(row.get('Aggregate rating')), datatype=XSD.float)))

    # Cost
    if pd.notna(row.get(cost_col)):
        g.add((restaurant_uri, BASE.averageCostForTwo, Literal(float(row.get(cost_col)), datatype=XSD.float))) 

    # Booleans
    if pd.notna(row.get('Has Online delivery')):
        has_delivery = str(row.get('Has Online delivery')).strip().lower() in ['yes', 'true', '1']
        g.add((restaurant_uri, BASE.hasOnlineDelivery, Literal(has_delivery, datatype=XSD.boolean)))
    if pd.notna(row.get('Has Table booking')):
        has_booking = str(row.get('Has Table booking')).strip().lower() in ['yes', 'true', '1']
        g.add((restaurant_uri, BASE.hasTableBooking, Literal(has_booking, datatype=XSD.boolean)))

    # Cuisines using detected cuisines col
    cuisines_text = clean(row.get(cuisines_col, ''))
    for cuisine in extract_cuisines(cuisines_text):
        cuisine_uri = cuisine_uris.get(cuisine)
        if not cuisine_uri:
            cuisine_uri = BASE[f"cuisine_{cuisine_counter}"]
            g.add((cuisine_uri, RDF.type, BASE.Cuisine))
            g.add((cuisine_uri, SCHEMA.name, Literal(cuisine)))
            cuisine_uris[cuisine] = cuisine_uri
        g.add((restaurant_uri, BASE.servesCuisine, cuisine_uri))

print(f"Converted {len(restaurants_df)} restaurants")
print(f"Found {len(cuisine_uris)} unique cuisines")
print(f"Graph now: {len(g)} triples")

In [ ]:
# Save KG
output_file = os.path.join(OUTPUT_DIR, "KEN4256-structured-KG-team2.ttl")
g.serialize(destination=output_file, format="turtle")

print("Task 2:")
print(f"Output location: {output_file}")
print(f"Total triples: {len(g):,}")
print(f"Recipe <-> Nutrition: {linked}/{len(nutrition_df)} linked")
print(f"Recipe <-> Restaurant: via {len(cuisine_uris)} shared cuisines")

Task3

In [ ]:
REVIEWS_TXT ="data/Reviews.txt"
STRUCTURED_KG = "outputs/KEN4256-structured-KG-team2.ttl"
SCHEMA_FILE = "outputs/schema.ttl"
OUTPUT_DIR = "outputs"
BASE_URI = "http://kg-course.io/food-nutrition/"
REVIEW_LIMIT = 1000
WIKIDATA_RECIPE_LIMIT = 1000

import spacy
nlp = spacy.load("en_core_web_sm")
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

categories = {
    "fruit": "wd:Q3314483",
    "vegetable": "wd:Q11004",
    "meat": "wd:Q10976",
    "fish": "wd:Q205149",
    "milk": "wd:Q10979",
}

all_ingredients_names = set()

for name, cat in categories.items():
    sparql.setQuery(f"""
    SELECT DISTINCT ?ingredientLabel WHERE {{
        ?ingredient wdt:P31/wdt:P279* {cat}.
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 2000
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    ingredients = [item['ingredientLabel']['value'].lower() for item in results["results"]["bindings"]]
    all_ingredients_names.update(ingredients)

all_ingredients_list = sorted(all_ingredients_names)

print(f"Total ingredients found: {len(all_ingredients_names)}")
print("Example ingredients:", all_ingredients_list[:20])

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
import pandas as pd

file_path = REVIEWS_TXT

# Load the dataset
df = pd.read_csv(file_path, sep="\t")

# Drop rows where 'Review' is NaN
df = df.dropna(subset=['Review'])

# Remove rows where 'Review' is empty or contains only whitespace
df = df[df['Review'].str.strip() != ""]

# Select the first 1,000 reviews after cleaning
subsection = df['Review'].iloc[0:1000]

# Optional: also get corresponding RecipeId
recipe_ids = df['RecipeId'].iloc[0:1000]

print(f"Total reviews after cleaning: {len(df)}")
print(f"Selected {len(subsection)} reviews for sentiment analysis")
print(subsection.head())

In [ ]:
def extract_ingredients(text, all_ingredients_names):
    doc = nlp(text[:500])  # For speed
    ingredients = set()

    # Named entities
    for ent in doc.ents:
        if ent.label_ in ['PRODUCT', 'ORG']:
            ingredient = ent.text.lower().strip()
            if len(ingredient) > 2:
                ingredients.add(ingredient)

    # Common food nouns
    food_words = {
        'chicken', 'beef', 'pork', 'fish', 'rice', 'pasta', 'cheese',
        'tomato', 'onion', 'garlic', 'pepper', 'salt', 'sugar',
        'flour', 'egg', 'milk', 'butter', 'oil', 'bread',
        'potato', 'carrot'
    }

    # Merge previously found ingredients properly
    food_words.update(all_ingredients_names)

    for token in doc:
        word = token.text.lower().strip()
        if word in food_words:
            ingredients.add(word)

    # Return unique list (set already ensures uniqueness)
    return list(ingredients)

In [ ]:
sentiment_labels = []
confidences = []
review_data = []

# Loop over reviews
for idx, text in enumerate(subsection):
    ingredients = extract_ingredients(text,all_ingredients_names)
    print(ingredients)
    inputs = tokenizer(text[:512], return_tensors="pt")

    # Run model
    with torch.no_grad():
        logits = model(**inputs).logits

    # Convert logits to probabilities
    probs = torch.softmax(logits, dim=1)

    # Predicted class
    predicted_class_id = logits.argmax().item()
    label = model.config.id2label[predicted_class_id]
    confidence = probs[0][predicted_class_id].item()

    # Make sentiment score signed
    if label.lower() == "positive":
        sentiment_score = confidence
    else:
        sentiment_score = -confidence

    # Append to lists
    sentiment_labels.append(label)
    confidences.append(confidence)
    review_data.append({
        'id': recipe_ids.iloc[idx],
        'text': text,
        'ingredients': ingredients,
        'sentiment_score': sentiment_score,
        'sentiment_label': label,
        'confidence': confidence
    })

In [ ]:
def extract_ingredients(text, all_ingredients_names):
    doc = nlp(text[:500])  # For speed
    ingredients = set()

    # Named entities
    for ent in doc.ents:
        if ent.label_ in ['PRODUCT', 'ORG']:
            ingredient = ent.text.lower().strip()
            if len(ingredient) > 2:
                ingredients.add(ingredient)

    # Common food nouns
    food_words = {
        'chicken', 'beef', 'pork', 'fish', 'rice', 'pasta', 'cheese',
        'tomato', 'onion', 'garlic', 'pepper', 'salt', 'sugar',
        'flour', 'egg', 'milk', 'butter', 'oil', 'bread',
        'potato', 'carrot'
    }

    # Merge previously found ingredients properly
    food_words.update(all_ingredients_names)

    for token in doc:
        word = token.text.lower().strip()
        if word in food_words:
            ingredients.add(word)

    # Return unique list (set already ensures uniqueness)
    return list(ingredients)


In [ ]:
g_unstructured = Graph()
g_unstructured.parse(SCHEMA_FILE, format="turtle") # Schema

ingredient_uris = {} # Avoid duplicates
ingredient_counter = 0

for i, review in enumerate(review_data):
    review_uri = BASE[f"review_{i}"]
    g_unstructured.add((review_uri, RDF.type, BASE.Review))
    g_unstructured.add((review_uri, BASE.hasSentimentScore,
                        Literal(review['sentiment_score'], datatype=XSD.float)))
    g_unstructured.add((review_uri, BASE.hasSentimentLabel,
                        Literal(review['sentiment_label'], datatype=XSD.string)))
    g_unstructured.add((review_uri, BASE.hasConfidence,
                        Literal(review['confidence'], datatype=XSD.float)))

    # Add ingredients
    for ingredient_name in review['ingredients']:
        # print(ingredient_name) # check
        if ingredient_name not in ingredient_uris:
            ingredient_uri = BASE[f"ingredient_{ingredient_counter}"]
            g_unstructured.add((ingredient_uri, RDF.type, BASE.Ingredient))
            g_unstructured.add((ingredient_uri, RDFS.label, Literal(ingredient_name)))
            ingredient_uris[ingredient_name] = ingredient_uri
            ingredient_counter += 1
        else:
            ingredient_uri = ingredient_uris[ingredient_name]

        g_unstructured.add((review_uri, BASE.mentionsIngredient, ingredient_uri)) ## Link review to ingredient
print(len(ingredient_uris))
print(f"Unstructured KG: {len(g_unstructured)} triples")

In [ ]:
# Query Wikidata for common ingredients
from rdflib import URIRef

def get_wikidata_id(ingredient_name):

    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setReturnFormat(JSON)
    query = f"""
    SELECT ?item WHERE {{
      ?item rdfs:label "{ingredient_name}"@en .
      ?item wdt:P31/wdt:P279* wd:Q2095 .  # instance of food
    }}
    LIMIT 1
    """

    try:
        sparql.setQuery(query)
        results = sparql.query().convert()
        if results['results']['bindings']:
            return results['results']['bindings'][0]['item']['value']
    except:
        pass # It might time out
    return None

# Link top 20 ingredients (else too many queries)
linked_count = 0
for ingredient_name in list(ingredient_uris.keys()):
    wikidata_uri = get_wikidata_id(ingredient_name)
    if wikidata_uri:
        ingredient_uri = ingredient_uris[ingredient_name]
        g_unstructured.add((ingredient_uri, OWL.sameAs, URIRef(wikidata_uri)))
        linked_count += 1

print(f"{linked_count} ingredients llinked")
print(f"Final unstructured KG has {len(g_unstructured)} triples")

# Save
unstructured_file = os.path.join(OUTPUT_DIR, "KEN4256-unstructured-KG-team2.ttl")
g_unstructured.serialize(destination=unstructured_file, format="turtle")
print(f"Saved to: {unstructured_file}")

# Create integrated KG (structured + unstructured)
g_integrated = Graph()
g_integrated.parse(STRUCTURED_KG, format="turtle")
g_integrated.parse(unstructured_file, format="turtle")

integrated_file = os.path.join(OUTPUT_DIR, "KEN4256-integrated-KG-team2.ttl")
g_integrated.serialize(destination=integrated_file, format="turtle")
print(f"Saved to {integrated_file}")

print(f"Unstructured KG: {len(g_unstructured):,} triples")
print(f"Integrated KG: {len(g_integrated):,} triples")
print("")
print("Stats:")
print(f"{len(review_data)} reviews")
print(f"{len(ingredient_uris)} unique ingredients")
print(f"{linked_count} Wikidata links")

Task 4

In [ ]:
INTEGRATED_KG = os.path.join(OUTPUT_DIR, "KEN4256-integrated-KG-team2.ttl") 
OUTPUT_DIR = "outputs"
from rdflib import Graph
import datetime

g = Graph()
g.parse(INTEGRATED_KG, format="turtle")
print(f"{len(g):,} triples")

def run_query(query, title): # Execute SPARQL query and return results
    results = g.query(query)
    print(f"\n{title}\n")
    
    result_list = []
    for row in results:
        result_list.append(row)
        print(" | ".join(str(val) for val in row))
    
    print(f"\n{len(result_list)} result(s)")
    return result_list, query

In [ ]:
query_1 = """
PREFIX : <http://kg-course.io/food-nutrition/>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?recipeName
WHERE {
    ?recipe a :Recipe ;
            schema:name ?recipeName ;
            schema:recipeIngredient ?ingredients .
    
    FILTER(CONTAINS(LCASE(STR(?ingredients)), "mango"))
}
ORDER BY ?recipeName
LIMIT 20
"""

results_1, _ = run_query(query_1, "4.1: Recipes wigth mango")

In [ ]:
query_2 = """
PREFIX : <http://kg-course.io/food-nutrition/>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?recipeName ?cookTime ?keywords
WHERE {
    ?recipe a :Recipe ;
            schema:name ?recipeName ;
            schema:cookTime ?cookTime ;
            schema:keywords ?keywords .
    
    # Contains 'pie' AND 'healthy'
    FILTER(CONTAINS(LCASE(?recipeName), "pie") || CONTAINS(LCASE(?keywords), "pie"))
    FILTER(CONTAINS(LCASE(?keywords), "healthy"))
    
    FILTER( # Check for PT formats with hours < 2 or only minutes
        CONTAINS(?cookTime, "PT") && 
        (!CONTAINS(?cookTime, "H") || 
         CONTAINS(?cookTime, "PT0") || 
         CONTAINS(?cookTime, "PT1H"))
    )
}
ORDER BY ?recipeName
LIMIT 10
"""

results_2, _ = run_query(query_2, "4.2: Healthy pies < 2h cooktime")

In [ ]:
query_3 = """
PREFIX : <http://kg-course.io/food-nutrition/>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?restaurantName ?address
WHERE {
    ?restaurant a :Restaurant ;
                schema:name ?restaurantName ;
                schema:addressLocality ?city ;
                :hasOnlineDelivery ?delivery ;
                :servesCuisine ?cuisine .
    
    ?cuisine schema:name ?cuisineName .
    
    OPTIONAL { ?restaurant schema:address ?address . }
    FILTER(LCASE(STR(?city)) = "new delhi")
    FILTER(CONTAINS(LCASE(STR(?cuisineName)), "chinese"))
    FILTER(?delivery = true)
}
ORDER BY ?restaurantName
LIMIT 20
"""

results_3, _ = run_query(query_3, "4.3: New Delhi chinese restaurants with delivery")

In [ ]:
query_4 = """
PREFIX : <http://kg-course.io/food-nutrition/>
PREFIX schema: <http://schema.org/>

SELECT (AVG(?cost) AS ?avgCost)
WHERE {
    ?restaurant a :Restaurant ;
                schema:addressLocality ?city ;
                :averageCostForTwo ?cost ;
                :servesCuisine ?cuisine .
    
    ?cuisine schema:name ?cuisineName .
    
    FILTER(CONTAINS(LCASE(STR(?city)), "davenport"))
    
    # Asian cuisines
    FILTER(
        CONTAINS(LCASE(STR(?cuisineName)), "indian") ||
        CONTAINS(LCASE(STR(?cuisineName)), "sushi") ||
        CONTAINS(LCASE(STR(?cuisineName)), "asian") ||
        CONTAINS(LCASE(STR(?cuisineName)), "chinese") ||
        CONTAINS(LCASE(STR(?cuisineName)), "thai")
    )
}
"""

results_4, _ = run_query(query_4, "4.4: avg cost Davenport (asian food)")

In [ ]:
query_5 = """
PREFIX : <http://kg-course.io/food-nutrition/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?recipeName ?calories ?image ?datePublished
WHERE {
    ?recipe a :Recipe ;
            schema:name ?recipeName ;
            schema:recipeCategory ?category ;
            schema:keywords ?keywords ;
            :hasNutrition ?nutrition .
    
    ?nutrition schema:calories ?calories .
    
    OPTIONAL { ?recipe schema:image ?image . }
    OPTIONAL { ?recipe schema:datePublished ?datePublished . }
    
    FILTER(CONTAINS(LCASE(?category), "dessert"))
    FILTER(CONTAINS(LCASE(?keywords), "easy"))
    FILTER(?calories < 300)
    
    # After 2000 (check dates)
    FILTER(
        !BOUND(?datePublished) ||
        (CONTAINS(STR(?datePublished), "200") ||
         CONTAINS(STR(?datePublished), "201") ||
         CONTAINS(STR(?datePublished), "202"))
    )
}
ORDER BY ?calories
LIMIT 5
"""

results_5, _ = run_query(query_5, "4.5: Top 5 easy desserts < 300 cal(after 2000)")

In [ ]:
query_6 = """
PREFIX : <http://kg-course.io/food-nutrition/>
PREFIX schema: <http://schema.org/>
SELECT ?recipeName ?avgSentiment ?prepTime ?sugar
WHERE {
    ?recipe a :Recipe ;
            schema:name ?recipeName ;
            schema:recipeCategory ?category ;
            :hasNutrition ?nutrition .
    OPTIONAL { ?recipe schema:prepTime ?prepTime . }
    OPTIONAL { ?nutrition schema:sugarContent ?sugar . }
    
    # Avg sentiment from reviews
    {
        SELECT ?recipe (AVG(?score) AS ?avgSentiment)
        WHERE {
            ?recipe :hasReview ?review .
            ?review :hasSentimentScore ?score .
        }
        GROUP BY ?recipe
    }
    
    FILTER(CONTAINS(LCASE(?category), "beverage") || CONTAINS(LCASE(?category), "drink")) # Category
}
ORDER BY DESC(?avgSentiment)
LIMIT 10
"""

results_6, _ = run_query(query_6, "4.6: Top 10 highly rated beverages")

In [ ]:
query_7 = """
PREFIX : <http://kg-course.io/food-nutrition/>
PREFIX schema: <http://schema.org/>

SELECT ?recipeName ?protein ?avgSentiment ?cuisineName ?usaCount
       (IF(?usaCount >= 5, true, false) AS ?cuisineCommonInUSA)
WHERE {
    ?recipe a :Recipe ;
            schema:name ?recipeName ;
            :hasNutrition ?nutrition ;
            :hasCuisine ?cuisine .

    ?nutrition schema:proteinContent ?protein .
    FILTER(?protein > 20)

    ?cuisine schema:name ?cuisineName .

    #"Highest-rated" recipes = average sentiment from reviews
    {
        SELECT ?recipe (AVG(?score) AS ?avgSentiment)
        WHERE {
            ?recipe :hasReview ?review .
            ?review :hasSentimentScore ?score .
        }
        GROUP BY ?recipe
    }


    BIND(COALESCE(?usaRestaurantCount, 0) AS ?usaCount)
}
ORDER BY DESC(?avgSentiment) DESC(?protein)
LIMIT 10
"""

results_7, _ = run_query(query_7, "4.7: Highest-rated protein-rich recipes + cuisine availability in USA")

In [ ]:
query_8 = """
PREFIX : <http://kg-course.io/food-nutrition/>
PREFIX schema: <http://schema.org/>
SELECT ?recipeName ?nds ?avgSentiment ?restaurantRating ?cuisineName
WHERE {
    ?recipe a :Recipe ;
            schema:name ?recipeName ;
            :hasNutrition ?nutrition ;
            :hasCuisine ?cuisine .
    
    ?nutrition schema:proteinContent ?protein ;
               schema:fiberContent ?fiber ;
               schema:sugarContent ?sugar .
    
    ?cuisine schema:name ?cuisineName .
    
    # Calculate NDS: (1.0 × Protein) + (1.5 × Fiber) − (2.0 × Sugar)
    BIND((1.0 * ?protein) + (1.5 * ?fiber) - (2.0 * ?sugar) AS ?nds)
    # Avg sentiment from reviews
    OPTIONAL {
        {
            SELECT ?recipe (AVG(?score) AS ?avgSentiment)
            WHERE {
                ?recipe :hasReview ?review .
                ?review :hasSentimentScore ?score .
            }
            GROUP BY ?recipe
        }
    }
    FILTER(?nds > 0)
}
ORDER BY DESC(?nds)
LIMIT 5
"""

results_8, _ = run_query(query_8, "4.8: Top 5 healthiest recipes by NDS")

In [ ]:
# Collate
all_queries = [
    ("4.1", query_1, results_1),
    ("4.2", query_2, results_2),
    ("4.3", query_3, results_3),
    ("4.4", query_4, results_4),
    ("4.5", query_5, results_5),
    ("4.6", query_6, results_6),
    ("4.7", query_7, results_7),
    ("4.8",  query_8, results_8),
]

output_file = os.path.join(OUTPUT_DIR, "SPARQL_query_results.txt")
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("Task 4:\n")
    for num, query, results in all_queries:
        f.write(f"\nQuery {num}\n")
        f.write("\nSPARQL Query:\n")
        f.write(query.strip() + "\n\n")
        f.write("\nResults:\n")        
        if results:
            for i, row in enumerate(results, 1):
                f.write(f"{i}. " + " | ".join(str(val) for val in row) + "\n")
            f.write(f"\nTOTAL: {len(results)} result(s)\n")
        else:
            f.write("No results found.\n")
        f.write("\n")

# for individual files
for num, query, results in all_queries:
    per_query_file = os.path.join(OUTPUT_DIR, f"query_{num.replace('.', '_')}.txt")
    with open(per_query_file, 'w', encoding='utf-8') as f:
        f.write("Task 4:\n")
        f.write(f"\nQuery {num}\n")
        f.write("\nSPARQL Query:\n")
        f.write(query.strip() + "\n\n")
        f.write("\nResults:\n")
        if results:
            for i, row in enumerate(results, 1):
                f.write(f"{i}. " + " | ".join(str(val) for val in row) + "\n")
            f.write(f"\nTOTAL: {len(results)} result(s)\n")
        else:
            f.write("No results found.\n")
        f.write("\n")

print(f"Task 4 - combined results saved toL {output_file}")
print(f"\nNo. of results for each query:")
for num, _, results in all_queries:
    print(f"Query #{num}: {len(results)}")